# POS-Tagging
На этом семинаре мы рассмотрим некоторые библиотеки, позволяющие делать морфологический анализ, и даже обучим свой теггер на основе [BERT](https://habr.com/ru/post/436878/) (stay tuned!)

# NLTK
Разнообразные теггеры находятся в модуле `nltk.tag`. Можно скачивать дополнительные модели (вспомним, как это делается?)

In [ ]:
#!pip install nltk

In [ ]:
import nltk

In [ ]:
nltk.download('brown')

In [ ]:
import nltk.tag
from nltk.corpus import brown

In [ ]:
brown.tagged_sents()

In [ ]:
# корпуса делятся на части - categories
print(brown.categories())

Идея 1 : давайте приписывать всем словам один самый частый тег. Его легко найти с помощью `nltk`.

In [ ]:
tags = [tag for (word, tag) in brown.tagged_words(categories='news')]
nltk.FreqDist(tags).max()

In [ ]:
default_tagger = nltk.tag.DefaultTagger('NN')
sentence = brown.sents()[5]
print(sentence)
default_tagger.tag(sentence)

Не очень здорово, верно? Давайте хранить самые частые слова и приписывать им их самый частый тег (а остальным пока приписывать `NN`)

In [ ]:
fd = nltk.FreqDist(brown.words(categories='news'))
cfd = nltk.ConditionalFreqDist(brown.tagged_words(categories='news'))
most_freq_words = fd.most_common(1000)
likely_tags = dict((word, cfd[word].max()) for (word, _) in most_freq_words)
unigram_tagger = nltk.UnigramTagger(model=likely_tags, backoff=default_tagger)
unigram_tagger.tag(sentence)

In [ ]:
# Можно легко оценить на каких-нибудь предложениях из того же корпуса
unigram_tagger.evaluate(brown.tagged_sents(categories='reviews'))

In [ ]:
# Можно просто приписывать самый частый тег всем словам из корпуса
unigram_tagger = nltk.UnigramTagger(brown.tagged_sents(categories='news'))

## Задание
Используя [документацию](http://www.nltk.org/api/nltk.tag.html#nltk.tag.sequential.BigramTagger) и функцию `help()` обучите биграммный теггер. Сравните его с униграммным на примерах, где должно стать лучше (выбор тега зависит от контекста), например: `They wind back the clock.` vs. `The wind rises.`

In [ ]:
### YOUR CODE HERE

Помимо этого в NLTK реализовано множество теггеров:
* regexp tagger
* ngram tagger
* Brill tagger
* CRF tagger
* etc.

## spaCy
[SpaCy](https://spacy.io/) — библиотека для обработки текстов, содержащая разные модули (токенайзер, pos-tagging, NER etc.)
Разработана специально для построения пайплайнов, сейчас нас будет интересовать токенизация и морфологический анализ.
Недостаток — некоторые модели (например, для NER) нужно загружать отдельно.

In [ ]:
#!pip install spacy

In [ ]:
#!python -m spacy download en_core_web_sm

In [ ]:
nltk.download('webtext')
from nltk.corpus import webtext

In [ ]:
first_text_id = webtext.fileids()[0]
sentences = webtext.raw(first_text_id).split('\r\n')
text = " ".join(sentences[:10])

In [ ]:
print(text)

In [ ]:
import spacy

# Загружаем весь пайплайн для английского
nlp = spacy.load("en_core_web_sm")

# Обрабатываем текст
doc = nlp(text)

# Выведем токены, леммы и теги
for i, s in enumerate(doc.sents):
    print("\n-- Sentence %d --" % i)
    for t in s:
        print(t.text, t.lemma_, t.pos_, sep="\t")

## Самостоятельно:
возьмите любой текст на английском языке (например, скопируйте абзац из Википедии), примените к нему пайплайн spaCy, 
выведите все биграммы, состоящие из прилагательного и существительного.

**Бонус-трек**: перепишите ваш код в виде функции, которая принимает на вход текст и последовательность тегов, а возвращает энграммы, соответствующие данной последовательности. Протестируйте функцию на 3/4-граммах.

In [ ]:
text = """
"""
### YOUR CODE HERE ###

# Flair
[Flair](https://github.com/zalandoresearch/flair) — библиотека для работы с векторными представлениями слов, содержит компоненты для решения 
разных задач NLP (sequence tagging, classification ...). Есть как готовые модели, так и возможность обучения на своих данных, дообучение предобученные векторных представлений и т.д.

Сегодня рассмотрим некоторые возможности flair для морфологического анализа на примере англоязычных данных.

In [ ]:
# !pip install flair

In [ ]:
# импортируем простейшие объекты
from flair.data import Sentence
from flair.models import SequenceTagger

# создаем предложение
sentence = Sentence('Moscow is the capital of Russia .')

# загружаем модель теггинга
tagger = SequenceTagger.load('pos')

# обрабатываем предложение
# ВАЖНО: объект `sentence` при этом меняется
tagger.predict(sentence)

print(sentence)
print(sentence.to_tagged_string())

In [ ]:
# если интересно узнать подробнее об этих объектах
help(sentence)

У flair есть встроенные датасеты (`flair.datasets`), можно попробовать использовать их.

In [ ]:
import flair.datasets
corpus = flair.datasets.UD_ENGLISH()

In [ ]:
corpus.test[0].to_tagged_string('upos')

In [ ]:
type(corpus.test[0])

In [ ]:
from random import choice
n = choice(range(len(corpus.test)))
sentence = corpus.test[n]
print(sentence.to_tagged_string('upos'))
print()

# помним, что теперь `sentence` поменяется
tagger.predict(sentence)

print(sentence)
print(sentence.to_tagged_string('upos'))

И наконец: **flair** позволяет обучать свои модели. Попробуем использовать эмбеддинги от Google, чтобы обучить свой теггер, вот [так](https://colab.research.google.com/drive/1OZN14wo1QGiwSpFhYuKPOPbzmJ2CzeWU).